# Part 1: Dataframe Construction
(for other parts of the assignment please scroll down and look for parts 2 and 3)

In [3]:
import pandas as pd

Let's load the Toronto neighborhood database as a table.

In [31]:
# The code was removed by Watson Studio for sharing.

In [32]:
Toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Drop "Not assigned" values in column Borough.

In [33]:
Toronto_df=Toronto_df[Toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
Toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check for empty cells or "Not assigned" values in Neighborhood column.

In [7]:
import numpy as np

print(len(Toronto_df[Toronto_df.Neighborhood == 'Not assigned']))
print(Toronto_df.Neighborhood.isnull().sum() )
print(len(Toronto_df[Toronto_df.Neighborhood == '']))

0
0
0


After droping "Not assigned" Borough there are no Neighborhoods which have no value or are valued "Not assigned".

Next let's check if there are duplicate values in Postal Code column.

In [8]:
print(len(set(Toronto_df['Postal Code'])) )
print(len(Toronto_df['Postal Code']) )

103
103


There are no duplicate values in Postal Code column. It seems that the case with "M5A" has been resolved already in Wikipedia page itself, as "M5A" already has two neighborhoods Regent Park, Harbourfront separated with a comma.
Let's  check if there is no duplicate neighbordhood names.

In [9]:
print(len(set(Toronto_df['Neighborhood'])) )

99


There are 4 duplicate neighborhood names. However, they have different postal codes, so essentially they are different neighborhoods. 
But to be able to distinguish between these neighborhoods, we have to create unique neighborhood name. We will achieve this by creating column which merges Postal code column and Neighborhood column.

In [34]:
Toronto_df['Neighborhood_PC']=Toronto_df['Postal Code']+' '+Toronto_df['Neighborhood']
Toronto_df.head()

,Postal Code,Borough,Neighborhood,Neighborhood_PC
0,M3A,North York,Parkwoods,M3A Parkwoods
1,M4A,North York,Victoria Village,M4A Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A Queen's Park, Ontario Provincial Government"


In [252]:
Toronto_df.shape

(103, 4)

Database has 103 rows.

# Part 2: Geographical coordinates
(for other parts of the assignment please scroll down and look for part 3, for part 1 scroll up)

Load the goegraphical coordinate database.

In [10]:

body = client_4ab6efedbd8545eeaf31c1dcdc1d649d.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-wiseierbro6z33',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
Geo_cord = pd.read_csv(body)
Geo_cord.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add the coordinate info to the neighborhood databse.

In [35]:
Toronto_df = Toronto_df.merge(Geo_cord, on='Postal Code')
Toronto_df.head()

,Postal Code,Borough,Neighborhood,Neighborhood_PC,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
Toronto_df.shape

(103, 5)

# Part 3: Clustering neighborhoods.
(for others parts of the assignment please scroll up and look for parts 1 and 2)

Let's retrieve location coordinates of Toronto.

In [36]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude



Install map rendering library.

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

Plot Toronto map with all neighborhoods with borough name, neighborhood name and postal code in the label.

In [37]:
import folium # map rendering library

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postal, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Postal Code'],Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, postal)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Load Foursqaure credentials.

In [43]:
CLIENT_ID = 'UNOJDCQH4GLAWEX422WINWFES5GAKQLLBSD135VKOTFPFZIL' #Foursquare ID
CLIENT_SECRET = '4CQFM1QAYYKXTM1MEJOIROCFFEHMUUWQFPZB01ADJNA1BJC0' #Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100 #maximum number of venues to retrieve

Create a function that allows to retrieve all venues in a 500m radius of a given location.

In [44]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood_PC', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Apply the given function to all Toronto neighborhoods and retrieve information on venues in the neighborhoods.

In [45]:
Toronto_venues = getNearbyVenues(names=Toronto_df['Neighborhood_PC'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

M3A Parkwoods
M4A Victoria Village
M5A Regent Park, Harbourfront
M6A Lawrence Manor, Lawrence Heights
M7A Queen's Park, Ontario Provincial Government
M9A Islington Avenue, Humber Valley Village
M1B Malvern, Rouge
M3B Don Mills
M4B Parkview Hill, Woodbine Gardens
M5B Garden District, Ryerson
M6B Glencairn
M9B West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
M1C Rouge Hill, Port Union, Highland Creek
M3C Don Mills
M4C Woodbine Heights
M5C St. James Town
M6C Humewood-Cedarvale
M9C Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
M1E Guildwood, Morningside, West Hill
M4E The Beaches
M5E Berczy Park
M6E Caledonia-Fairbanks
M1G Woburn
M4G Leaside
M5G Central Bay Street
M6G Christie
M1H Cedarbrae
M2H Hillcrest Village
M3H Bathurst Manor, Wilson Heights, Downsview North
M4H Thorncliffe Park
M5H Richmond, Adelaide, King
M6H Dufferin, Dovercourt Village
M1J Scarborough Village
M2J Fairview, Henry Farm, Oriole
M3J Northwood Park, York University
M4J East Toronto

In [46]:
print(Toronto_venues.shape)
print(len(Toronto_venues['Venue Category'].unique()))
Toronto_venues.head()

(2127, 7)
272


,Neighborhood_PC,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,M3A Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In total in all neighborhoods 2127 venues were retrieved and 272 different venue categories were listed. <br>
Let's now see how many venues were retrieved by each Neighborhood.

In [51]:
Toronto_venues.groupby('Neighborhood_PC').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood_PC,,,,,,
"M1B Malvern, Rouge",1,1,1,1,1,1
"M1C Rouge Hill, Port Union, Highland Creek",1,1,1,1,1,1
"M1E Guildwood, Morningside, West Hill",8,8,8,8,8,8
M1G Woburn,5,5,5,5,5,5
M1H Cedarbrae,9,9,9,9,9,9
M1J Scarborough Village,2,2,2,2,2,2
"M1K Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5
"M1L Golden Mile, Clairlea, Oakridge",7,7,7,7,7,7
"M1M Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2


Now let's create a table with dummie variables showing what type of venue is listed in each row.

In [47]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood_PC'] = Toronto_venues['Neighborhood_PC'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood_PC,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now group them and see frquency of each type by neighborhood.

In [48]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood_PC').mean().reset_index()
Toronto_grouped.head()

,Neighborhood_PC,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"M1B Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"M1C Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"M1E Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
Toronto_grouped.shape

(100, 273)

Note that we have lost 3 rows. For 3 neighborhoods there were no venues to retrieve. We will deal with that leater. <br>
But now we will create a function that arranes the venues types in descening order by frequency.

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Get top 10 venues types for each neighborhood.

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_PC']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood_PC'] = Toronto_grouped['Neighborhood_PC']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted.head()

,Neighborhood_PC,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"M1B Malvern, Rouge",Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
1,"M1C Rouge Hill, Port Union, Highland Creek",Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Farmers Market
2,"M1E Guildwood, Morningside, West Hill",Electronics Store,Moving Target,Rental Car Location,Mexican Restaurant,Breakfast Spot,Bank,Intersection,Medical Center,Diner,Discount Store
3,M1G Woburn,Coffee Shop,Mexican Restaurant,Korean Restaurant,Convenience Store,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,M1H Cedarbrae,Hakka Restaurant,Fried Chicken Joint,Lounge,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Bakery,Dog Run


Next step is to cluster the neighborhoods based on frequency of venues in nieghborhoods.

In [58]:
from sklearn.cluster import KMeans 

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood_PC', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 2, 2, 4, 2, 2, 2, 2], dtype=int32)

Now added cluster labels to original table. We add 1 to cluster labels so to create space for the neighborhoods which did not have any venues retrieved from Foursqaure (they were not used in cluster analysis). 0 label later will be assigned to the "empty" neighborhoods. <br>
Then merge original neighborhood table with the venues table. Using left join we assure that the "empty" neighborhoods will be assigned 'NaN' values.

In [124]:
# add clustering labels
#Toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_+1)

Toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.merge(Toronto_venues_sorted.set_index('Neighborhood_PC'), on='Neighborhood_PC',how='left')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Neighborhood_PC,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Bus Stop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Event Space,Electronics Store
1,M4A,North York,Victoria Village,M4A Victoria Village,43.725882,-79.315572,3.0,Hockey Arena,Portuguese Restaurant,Coffee Shop,Pizza Place,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Yoga Studio,Electronics Store,Ice Cream Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Clothing Store,Furniture / Home Store,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Coffee Shop,Vietnamese Restaurant,Concert Hall,Dim Sum Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Coffee Shop,Sushi Restaurant,Creperie,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café


Assign 0s to "empty" neighborhoods.

In [126]:
where_are_NaNs = np.isnan(Toronto_merged['Cluster Labels'])
Toronto_merged.loc[where_are_NaNs,'Cluster Labels']=0

In [128]:
Toronto_merged=Toronto_merged.astype({'Cluster Labels':'int32'})

Plot clustered neighborhoods and assign specific color to each neighborhood.

In [131]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine the clusters.

##### Cluster 1

In [134]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,North York,"Willowdale, Newtonbrook",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Scarborough,Upper Rouge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These neighborhoods did not have any venues listed in Fouraquare. We call in a simple way "Empty neighborhoods".

##### Cluster 2

In [135]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,"Rouge Hill, Port Union, Highland Creek",1,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Farmers Market


The smallest cluster, it lists only one neighborhood. It seems that is defined by the number 1 venue - Bar, since other venues are quite common in other clusters too. We name it "Outgoing neighborhood".

##### Cluster 3

In [136]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,"Humberlea, Emery",2,Baseball Field,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Farmers Market
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2,Baseball Field,Pool,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


This is the second smallest cluster. It has Baseball field on the top, next is Yoga Studio. One of neighborhoods also have a pool. It will be called "Sporty Neighborhood" cluster.

##### Cluster 4

In [137]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,3,Hockey Arena,Portuguese Restaurant,Coffee Shop,Pizza Place,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,"Regent Park, Harbourfront",3,Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Yoga Studio,Electronics Store,Ice Cream Shop
3,North York,"Lawrence Manor, Lawrence Heights",3,Clothing Store,Furniture / Home Store,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Coffee Shop,Vietnamese Restaurant,Concert Hall,Dim Sum Restaurant
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",3,Coffee Shop,Sushi Restaurant,Creperie,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
6,Scarborough,"Malvern, Rouge",3,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
7,North York,Don Mills,3,Gym / Fitness Center,Café,Japanese Restaurant,Caribbean Restaurant,Baseball Field,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center
8,East York,"Parkview Hill, Woodbine Gardens",3,Pizza Place,Gym / Fitness Center,Intersection,Athletics & Sports,Gastropub,Fast Food Restaurant,Bank,Pharmacy,Pet Store,Breakfast Spot
9,Downtown Toronto,"Garden District, Ryerson",3,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Japanese Restaurant,Cosmetics Shop,Fast Food Restaurant,Ramen Restaurant
10,North York,Glencairn,3,Park,Pizza Place,Pub,Japanese Restaurant,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
13,North York,Don Mills,3,Coffee Shop,Asian Restaurant,Gym,Beer Store,Restaurant,Sandwich Place,Bike Shop,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop


This cluster by far is the largest with 82 members. In the most of neighborhoods one of the most common venues is coffee shop. Other venues frequently in the top include Gym, Grocery store, Clothing store. In general neighborhoods in this cluster have wide variety of venues. We may call this cluster "Standard cosy neighborhood".

##### Cluster 5

In [138]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,4,Park,Food & Drink Shop,Bus Stop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Event Space,Electronics Store
21,York,Caledonia-Fairbanks,4,Park,Pool,Women's Store,Greek Restaurant,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
35,East York,"East Toronto, Broadview North (Old East York)",4,Park,Metro Station,Convenience Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store,Doner Restaurant
45,North York,"York Mills, Silver Hills",4,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
61,Central Toronto,Lawrence Park,4,Park,Bus Line,Swim School,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Department Store
64,York,Weston,4,Park,Convenience Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
66,North York,York Mills West,4,Convenience Store,Park,Construction & Landscaping,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
85,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",4,Park,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
91,Downtown Toronto,Rosedale,4,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",4,Park,River,College Rec Center,College Stadium,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


This is second largest cluster with 10 entries. It's clear venue leader is Park. The most of neighborhoods also include Diner, Dim Sum Restaurant, Distribution Center, Discount store. The cluster could be called "The green dining neighborhood".

##### Cluster 6

In [139]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,"Islington Avenue, Humber Valley Village",5,Pizza Place,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
32,Scarborough,Scarborough Village,5,Pizza Place,Playground,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
50,North York,Humber Summit,5,Home Service,Pizza Place,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
63,York,"Runnymede, The Junction North",5,Brewery,Pizza Place,Bus Line,Convenience Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
77,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",5,Mobile Phone Shop,Bus Line,Pizza Place,Sandwich Place,Yoga Studio,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center


Here the top venues are Pizza Place, Yoga Studio. We may call it "Relaxed pizza lovers' neighborhood"